# METODI DIRETTI
## Risoluzione di sistemi lineari con matrice generica
### Scrivere uno script Python che:
1. Crea un problema test di dimensione variabile $n$ la cui soluzione esatta sia il vettore $x$ di tutti elementi unitari e $b$ il termine noto ottenuto moltiplicando la matrice $A$ per la soluzione $x$.
2. calcola il numero di condizione (o una stima di esso)
3. risolve il sistema lineare $Ax = b$ con la **fattorizzazione LU con pivoting**.

### Problemi test
- Una matrice di numeri casuali $A$ generata con la funzione `randn` di Matlab, ($n$ variabile fra 10 e 1000)

In [36]:
import numpy as np
import scipy.linalg as LA

for n in range(10, 1000):
    A = np.random.randn(n, n)
    x = np.ones((n, 1))
    b = np.dot(A, x)

    condA = np.linalg.cond(A)

    lu, piv = LA.lu_factor(A)

    my_x = LA.lu_solve((lu, piv), b)
    print('(', n, ') - norm =', LA.norm(x-my_x, 'fro'), sep='')


(10) - norm =5.10702591327572e-15
(11) - norm =4.7268780188129665e-14
(12) - norm =4.221768122166024e-15
(13) - norm =2.7058483469297603e-15
(14) - norm =4.191000011072726e-15
(15) - norm =4.253762688903432e-15
(16) - norm =3.85552819763498e-15
(17) - norm =6.74134880507727e-15
(18) - norm =1.148169227861664e-13
(19) - norm =5.738897349925092e-15
(20) - norm =1.5516136359034022e-14
(21) - norm =7.0461999135950884e-15
(22) - norm =5.7238430682505745e-15
(23) - norm =8.083308089220596e-15
(24) - norm =3.9950909281768e-14
(25) - norm =1.6138381524780443e-14
(26) - norm =5.209714888809006e-14
(27) - norm =2.45690624539368e-14
(28) - norm =2.3613846074421225e-14
(29) - norm =7.682229915320612e-15
(30) - norm =1.7403254977109662e-14
(31) - norm =1.785803311922223e-14
(32) - norm =2.044890973928333e-14
(33) - norm =1.8756198598693687e-14
(34) - norm =1.2976792799509356e-14
(35) - norm =4.113087877861829e-14
(36) - norm =2.423335857817079e-13
(37) - norm =1.4902237106265587e-14
(38) - norm =2.

KeyboardInterrupt: 

## Risoluzione di sistemi lineari con matrice simmetrica e definita positiva.
### Scrivere uno script Python che:
1. crea un problema test di dimensione variabile $n$ la cui soluzione esatta sia il vettore $x$ di tutti elementi unitari e $b$ il termine noto ottenuto moltiplicando la matrice $A$ per la soluzione $x$.
2. calcola il numero di condizione (o una stima di esso)
3. risolve il sistema lineare $Ax = b$ con la fattorizzazione di Cholesky.

### Problemi test
- matrice di Hilbert di dimensione n (con n variabile fra 2 e 15)
- matrice tridiagonale simmetrica e definita positiva avente sulla diagonale elementi uguali a 9 e quelli sopra e sottodiagonali uguali a -4.

### Per ogni problema test:
- Disegna il grafico del numero di condizione in funzione della dimensione del sistema
- Disegna il grafico dell’errore in norma 2 in funzione della dimensione del sistema

In [3]:
import numpy as np
import scipy.linalg as LA
import matplotlib.pyplot as plt

MIN, MAX = 2, 15

# Hilbert
K_A = np.zeros(MAX-MIN)
Err = np.zeros(MAX-MIN)
for n in range(MIN, MAX):
    A = LA.hilbert(n)
    x = np.ones((n, 1))
    b = np.dot(A, x)

    L = LA.cholesky(A, lower = True)
    y = LA.solve(L, b, lower = True)
    my_x = LA.solve(L.T, y, lower = False)

    K_A[n - MIN] = np.linalg.cond(A)
    Err[n - MIN] = LA.norm(x-my_x, ord = 2)
    x = np.arange(start=MIN, stop=MAX)

plt.title('CONDIZIONAMENTO (normale) di A e ERRORE (tratteggiato)')
plt.xlabel('dimensione matrice: n')
plt.xticks(x)
plt.ylabel('K(A) e Errore')
plt.plot(x, K_A)
plt.plot(x, Err, linestyle='dashed')
plt.show()

# Tridiagonale simmetrica e definita positiva
K_A = np.zeros(MAX-MIN)
Err = np.zeros(MAX-MIN)
for n in range(MIN, MAX):
    A = np.diag(np.full(n, fill_value = 9)) + np.diag(np.full(n-1, fill_value = -4), k = 1) + np.diag(np.full(n-1, fill_value = -4), k = -1)
    x = np.ones((n, 1))
    b = np.dot(A, x)

    L = LA.cholesky(A, lower = True)
    y = LA.solve(L, b, lower = True)
    my_x = LA.solve(L.T, y, lower = False)

    K_A[n - MIN] = np.linalg.cond(A)
    Err[n - MIN] = LA.norm(x-my_x, ord = 2)
    x = np.arange(start=MIN, stop=MAX)

plt.title('CONDIZIONAMENTO (normale) e NORMA (tratteggiato) di A')
plt.xlabel('dimensione matrice: n')
plt.xticks(x)
plt.ylabel('K(A)')
plt.plot(x, K_A)
plt.plot(x, Err, linestyle='dashed')
plt.show()



LinAlgError: 14-th leading minor of the array is not positive definite

# METODI ITERATIVI
Scrivi le funzioni `Jacobi(A, b, x0, maxit, tol, xTrue)` e `GaussSeidel(A, b, x0, maxit, tol, xTrue)` per implementare i metodi di Jacobi e di Gauss Seidel per la risoluzione di sistemi lineari con matrice a diagonale dominante.

In particolare:
- `x0` sia l’iterato iniziale;
- La condizione d’arresto sia dettata dal numero massimo di iterazioni consentite `maxit` e dalla tolleranza `tol` sulla differenza relativa fra due iterati successivi;
- Si preveda in input la soluzione esatta `xTrue` per calcolare l’errore relativo ad ogni iterazione.

Entrambe le funzioni restituiscano in output:
- La soluzione `x`;
- Il numero `k` di iterazioni effettuate;
- Il vettore `relErr` di tutti gli errori relativi.

Test
- Considerare la precedente matrice tridiagonale per `N = 100`.
- Verificare, calcolando il raggio spettrale della matrice, la convergenza dei metosi.
- Eseguire i metodi con tolleranza 1.e-8.
- Riportare in un grafico l’errore relativo (in norma 2) di entrambi i metodi al variare del numero di iterazioni per `N` fiossato (scegliere almeno due valori di `N`).
- Riportare in un grafico l’errore relativo finale dei metodi al variare della dimensione `N` del sistema.
- Riportare in un grafico il numero di iterazioni di entrambi i metodi al variare di `N`
- Riportare in un grafico il tempo impiegato dai metodi di Jacobi, Gauss Sidel, LU, Cholesky al variare di `N`.

In [ ]:
# TODO

# Traccia per la discussione
1. Utilizzando i grafici richiesti: spiegare l’andamento dell’errore rispetto al numero di condizione della
matrice, l’andamento del tempo di esecuzione rispetto alla dimensione del sistema in relazione alla
complessità computazioneale degli algoritmi utilizzati.
1. Discutere la differenza di errore e tempo di esecuzione ottenuti con i metodi diretti e iterativi.